In [ ]:
import matplotlib.pyplot as plt
import numpy

In [ ]:
class Population:
    growth_rate: float
    init: float
    value: float
    history: list[float]
    generations: int

    def __init__(self, r: float, X: list[float]):
        self.history = X
        self.growth_rate = r
        self.generations = len(X)
        self.init = X[0]
        self.value = X[self.generations - 1]

    # Generators

    @classmethod
    def new(cls, r: float, x0: float) -> 'Population':
        return Population(r = r, X = [x0])

    @classmethod
    def set_with(cls, r: float, history: list[float]) -> 'Population':
        return Population(r = r, X = history)
    
    def clone(self):
        return Population.set_with(r=self.growth_rate, history=self.history)
    
    # Behaviours

    def next_generation(self) -> 'Population':
        population = self.clone()
        r: float = population.growth_rate
        x: float = population.value
        x1 = r * x * (1 - x)

        X = population.history + [x1]
        population = Population.set_with(r=self.growth_rate, history=X)
        return population

    def next_nth_generations(self, n: int) -> 'Population':
        population = self.clone()
        N = n + population.generations
        return population.until_nth_generation(N)
    
    def until_nth_generation(self, N: int) -> 'Population':
        population = self.clone()
        while population.generations < N:
            population = population.next_generation()
        return population


In [ ]:
population = Population.new(r=1.0, x0 = 0.5)
population = Population.set_with(r=1.0, history = [0.5])

In [ ]:
population = population.next_generation()
population.__dict__

In [ ]:
population = population.until_nth_generation(15)
population.__dict__

In [ ]:
population = population.next_nth_generations(10)
population.__dict__

### Logistic differential equation

In [ ]:
def population_next_generation(growth_rate: float, population: float) -> float:
    r: float = growth_rate
    x: float = population
    return r * x * (1 - x)

def population_nth_generation(growth_rate: float, population_history: list, generations: int) -> list:
    r: float = growth_rate
    t: int = generations
    X: list = population_history
    x0: float = population_history[-1]

    x1 = population_next_generation(r, x0)
    X = X + [x1]
    t = t - 1
    
    if t > 0:
        return population_nth_generation(r, X, t)
    else:
        return X

### Plot functions

In [ ]:
def plot_population_over_time(population_history: list):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1) 

    x = range(len(population_history))
    y = population_history
        
    ax.cla()
    ax.plot(x, y)

def plot_convergence_values(list_population_histories: list[dict]):
    R = []
    C = []

    hists = list_population_histories

    for i in range(0, len(hists)):
        r = hists[i]['growth_rate']
        conv = hists[i]['convergence_value']
        R = R + [r]
        C = C + [conv]

    plt.scatter(R, C)

### Next generation population

In [ ]:
r = 2.5
x = 0.5

In [ ]:
x = population_next_generation(r, x)
x

### Calculate all generation's population

In [ ]:
r = 3.0
x = 0.5
n = 100

X = population_nth_generation(r, [x], n)
plot_population_over_time(X)

### Convergence value

In [ ]:
def convergence_value(series: list[float]) -> float:
    derivative_first = list( numpy.gradient( series ))
    derivative_second = list( numpy.gradient( series ))

    n: int = len(series) - 1
    value_nth = series[n]
    angle_nth = derivative_first[n]
    curvature_nth = derivative_second[n]

    if is_zero(angle_nth) and is_zero(curvature_nth):
        return value_nth
    else: 
        return None

def is_zero(value) -> bool:
    limit = 5e-7
    return limit > abs(value)
    

In [ ]:
r = 1.0
x = 0.5
n = 150
X = population_nth_generation(r, [x], n)

convergence_value(X)

## Find convergence value starting from x0

In [ ]:
def find_convergence(growth_rate: float, population_history: list, try_until: int = 5000):
    r: float = growth_rate
    X: list = population_history

    if len(X) < 10:
        X = population_nth_generation(r, [x], 10)

    convergence: float = None

    while convergence is None and len(X) < try_until:
        X = population_nth_generation(r, X, 1)
        convergence = convergence_value(X)

    # TODO - create struct
    return {
        'growth_rate': growth_rate,
        'initial_value': population_history[0],
        'convergence_value': convergence,
        'generation': len(X) - 1,
        'series': X,
    }

In [ ]:
r = 1.0
x = 0.5

convergence = find_convergence(r, [x], 5000)

print(convergence['convergence_value'])
print(convergence['generation'])
plot_population_over_time(convergence['series'])

### Build datatable r vs x vs n

In [ ]:
x = 0.5
r0 = 1.0
rN = 4.0
r_step = 0.1

results = []

for r in numpy.arange(r0, rN, r_step):
    result = find_convergence(r, [x])
    results = results + [result]

In [ ]:
plot_convergence_values(results)